# Essential Commands

### General Info
- After a linux system boots, press CTRL + ALT + F2 on the keyboard to access a terminal
- Consoles are actually terminal emulators

- Show IP Address Info
    ```shell
    ip addr
    ```

    Example Output:
    ```shell
    [azureadmin@centos01 squid]$ ip addr
    1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
        link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
        inet 127.0.0.1/8 scope host lo
          valid_lft forever preferred_lft forever
        inet6 ::1/128 scope host
          valid_lft forever preferred_lft forever
    2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
        link/ether 00:22:48:27:87:eb brd ff:ff:ff:ff:ff:ff
        inet 172.16.1.4/24 brd 172.16.1.255 scope global noprefixroute eth0
          valid_lft forever preferred_lft forever
        inet6 fe80::222:48ff:fe27:87eb/64 scope link
          valid_lft forever preferred_lft forever
    3: enP12806s1: <BROADCAST,MULTICAST,SLAVE,UP,LOWER_UP> mtu 1500 qdisc mq master eth0 state UP group default qlen 1000
        link/ether 00:22:48:27:87:eb brd ff:ff:ff:ff:ff:ff
    ```
- A 'pager' allows us to scroll through pages of output

### Read and Use System Documentation
- Most command have help support (--help, -h, /h, -?, etc.)
- Use man (manual pages) to view detailed help for a command
  - Example: man journalctl
  - Manual pages have categories or section
    - You can type `man man` to view the manual pages for manual pages :)
      - This will show you the sections available
- Apropos let's you search through man pages
  - apropos director will show man pages that have the word 'director'
  - The first time you run apropos, you will receive an error because you must first init the database. You can do that using this command:
    - `sudo mandb`
- Autocompletion will save you a lot of time when typing. 
  - You can press tab twice with some commands for autocompletion
- Man page sections (for quick reference). These can be viewed by typing `man man`
  ~~~shell
         The table below shows the section numbers of the manual followed by the types of pages they contain.

       1   Executable programs or shell commands
       2   System calls (functions provided by the kernel)
       3   Library calls (functions within program libraries)
       4   Special files (usually found in /dev)
       5   File formats and conventions eg /etc/passwd
       6   Games
       7   Miscellaneous (including macro packages and conventions), e.g. man(7), groff(7)
       8   System administration commands (usually only for root)
       9   Kernel routines [Non standard]
  ~~~

### SSH
`ssh -V` will show you the SSH Version

### Work with files and directories
- `ls` can be used to list the filesystem
- Hidden files begin with a '.'
  - `ls -a` will list all files, including hidden files
- `ls -l` will show the long listing format (additional details) of files in a directory
- `ls -lh` will show file sizes in human readable format. `-h` must be used with `-l`
- Linux file systems exist in a 'tree form', the root of the tree is '/'
- Use `cd` to change directories
- `touch` can be used to create a file


### Hard Links and Soft Links
- Inodes
  - An inode is a reference to a file on the disk
  - The `stat` command can be used to view inodes
  - When there are 0 links to an inode, the data itself is erased from the disk
  - Hard links point to the same inode
- You can only hardlink to files, not directories
- You cannot hardlink to files on a mount, only files on the same filesystem
- Soft links create a shortcut directly to the file, rather than a link to the inode
- `readlink` can be used to view the file behind a softlink
- broken soft links are highlighted in red in the output of `ls -l`


### File System Permissions
- The first permission setting in the output of the long listing on a file indicates the file type:
  - -: regular file
  - d: directory
  - l: symbolic link
  - p: pipe
  - s: socket
  - c: character device
  - b: block device
- Linux assigns default permissions to a file or directory when it is created. 
  - Default permissions are calculated based on the umask permission value subtracted from a preset initial permission value
    - The umask is a 3 digit octal value that refers to read, write, and execute permissions
    - The default umask for the root user is 0022
    - The default umask for standard users is 0002
    - To view the current umask, run the `umask` command without any options
    - The predefined initial permissions values are 666 for files and 777 for directories
    - If you modify the umask, it will be set backt to 0022/0002 when you logoff. You need to add it to the appropriate shell startup
      file. `~/.bash_rc` or `~/.bash_profile`
- Only the owner of a file or dir can change permissions (the exception is the super user)
- `chgrp` can be used to change the owning group of a file or directory
- `chown` can be used to change the owning user
- `groups` can be used to view what group memberships a user has
- `chmod` can be used to change permissions on a file or directory
- SUID, GID, Sticky Bit Permissions
  - SUID = Set UserId bit
  - When the SUID bit is set, the file is executed as the owner of the file, rather than the person running that file
  - ~~~shell
    [azureadmin@centos01 ~]$ touch suidfile
    [azureadmin@centos01 ~]$ ls suidfile
    suidfile
    [azureadmin@centos01 ~]$ ll suidfile
    -rw-rw-r--. 1 azureadmin azureadmin 0 Aug 28 17:46 suidfile
    [azureadmin@centos01 ~]$ chmod 4660 suidfile
    [azureadmin@centos01 ~]$ ls -l suidfile
    -rwSrw----. 1 azureadmin azureadmin 0 Aug 28 17:46 suidfile
    [azureadmin@centos01 ~]$
    ~~~

  - Find SUID files: `find . -perm /4000`
  - SGID = File is executed as the owning group of the file, rather than the person running the file
  - ~~~shell
    [azureadmin@centos01 ~]$ touch sgidfile
    [azureadmin@centos01 ~]$ chmod 2440 sgidfile
    [azureadmin@centos01 ~]$ ll sgidfile
    -r--r-S---. 1 azureadmin azureadmin 0 Aug 28 17:49 sgidfile
    [azureadmin@centos01 ~]$
    ~~~
  - Find GUID Files: `find . -perm /2000`

  - Sticky Bit
    - The sticky bit is typically set on public directories to inhibit file erasures by non-owners

  - ACLs
    - In addition to the standard UGO/RWX permission model, you can also apply ACLs to files and directories
    - ACLs define permissions for named users and named groups
    - ACLs are categorized into two groups, default ACLs and access ACLs
      - Access ACLs are set on individual files and directories
      - Default ACLs can only be applied at the directory level and are inherited by subdirectories and files
    - There are two commands to manage ACLs
      - `getfacl` and `setfacl`
      - 

### Manipulating File Content
- use cat to read a file to stdout
- use tac to read a file in reverse to stdout
- use sed to search and replace text in a file
  - Example: `sed 's/canda/canada/g' userinfo.txt`
  - without the 'g', sed only replaces the first occurence of the word in each line
  - use `-i` to replace in-line
- Use cut to extract content from a file
  - Example: `cut -d ' ' -f 1 userinfo.txt`
- `uniq` can be used to get unique entries in a file, but the entries must be sorted first
  - Example: `sort countries.txt | uniq`

### Archive and Backup Data
- Use `tar -tf myFile.tar` to view the contents of a tarball
- Use `tar cvf myFile.tar mydir` to archive a directory
- use `tar xvf myFile.tar` to extract the contents of a tarball

### Compress and Decompress Files
- gzip, bzip2, and xz can be used to compress/decompress files
- These compression utilities only compress a single file, which is why they are often combined with `tar`
  - `tar czf archive.tar.gz myFile` = Create gzip compressed tarball

### Backup files to a remote system
- use `rsync` to copy files to/from a remote system
- `dd` can be used to create an image of a disk
  - unmount the disk first
  - `sudo dd if=/dev/sda of=diskImage.raw bs=1M status=progress`
  - reverse `if` and `of` to restore an image

### Input/Output Redirection


### Search for files and directories
- the `find` command is a powerful tool that can be used to search for files and directories 
- the `locate` command is used to locate occurrences of a specified string as they appear in path names
  - `locate` searches `/var/lib/mlocate/mlocate.db` every time you run it, rather than searching the directory directly
  - the `mlocate.db` directory can be created or updated using the `updatedb` command

<hr />
## Commands to Know

##### System Commands
- hostnamectl
  
##### Help commands
- mandb
- man
- apropos
- which
- whereis
- whatis

##### File/Dir Commands
- ls
- mkdir
- cp
- stat
- mv
- cd
- pwd
- touch
- ln
- rm
- find
- locate
- updatedb
- getfacl
- setfacl

##### Text Manipulation Commands
- grep
- echo
- awk
  
##### Network Commands
- ip 
- ssh

##### Permission Related Commands
- chown
- chgrp
- chmod
- groups
- chmod

##### Read/Write Files
- cat
- tac
- more
- less
- tail
- head
- sed
- cut
- uniq
- sort
- diff 
- grep
- column 

##### Archive and Backup Data
- tar

##### Compress/Decompress Data
- gzip / gunzip
- bzip2 / bunzip
- xz / unxz
  
##### Backup Files
- rsync
- dd

##### System Information
uname
lscpu